# Communication Patterns in Backend Systems

Overview of key communication patterns for distributed systems and microservices.

## 1. Synchronous vs Asynchronous Communication

| Aspect | Synchronous | Asynchronous |
|--------|-------------|---------------|
| **Blocking** | Caller waits for response | Caller continues immediately |
| **Coupling** | Tight (both services must be available) | Loose (producer/consumer decoupled) |
| **Latency** | Accumulated across chain | Perceived latency reduced |
| **Use Case** | Real-time queries, CRUD ops | Background jobs, notifications |

```
SYNCHRONOUS                          ASYNCHRONOUS
┌────────┐      ┌────────┐           ┌────────┐    ┌───────┐    ┌────────┐
│ Client │─────▶│ Server │           │Producer│───▶│ Queue │───▶│Consumer│
│        │◀─────│        │           │        │    │       │    │        │
└────────┘      └────────┘           └────────┘    └───────┘    └────────┘
   │  wait...      │                     │ fire &      │           │
   │◀──────────────│                     │ forget      │  process  │
```

## 2. REST vs gRPC vs GraphQL

| Feature | REST | gRPC | GraphQL |
|---------|------|------|----------|
| **Protocol** | HTTP/1.1 (JSON) | HTTP/2 (Protobuf) | HTTP (JSON) |
| **Contract** | OpenAPI/Swagger | .proto files | Schema (SDL) |
| **Data Fetching** | Over/under-fetching common | Fixed response | Client specifies fields |
| **Streaming** | Limited (SSE) | Bidirectional | Subscriptions |
| **Performance** | Moderate | High (binary, multiplexed) | Moderate |
| **Browser Support** | Native | Requires grpc-web | Native |
| **Best For** | Public APIs, CRUD | Internal microservices | Flexible client needs |

```
REST                        gRPC                        GraphQL
┌────────┐                 ┌────────┐                  ┌────────┐
│ Client │                 │ Client │                  │ Client │
└───┬────┘                 └───┬────┘                  └───┬────┘
    │ GET /users/123           │ rpc GetUser()             │ query { user(id:123)
    │ POST /orders             │ rpc CreateOrder()         │   { name, email } }
    ▼                          ▼                           ▼
┌────────┐                 ┌────────┐                  ┌────────┐
│ Server │                 │ Server │                  │ Server │
│  JSON  │                 │Protobuf│                  │Resolver│
└────────┘                 └────────┘                  └────────┘
```

## 3. Message Queues

Decouples producers from consumers, enabling reliable async processing.

**Key Patterns:**
- **Point-to-Point**: One consumer processes each message (work queues)
- **Pub/Sub**: Message broadcast to all subscribers (fan-out)

```
POINT-TO-POINT (Work Queue)              PUB/SUB (Fan-out)
┌──────────┐                            ┌──────────┐
│ Producer │                            │ Producer │
└────┬─────┘                            └────┬─────┘
     │                                       │
     ▼                                       ▼
┌─────────────┐                        ┌──────────┐
│    Queue    │                        │ Exchange │ (topic/fanout)
│ ┌─┬─┬─┬─┬─┐ │                        └────┬─────┘
│ │●│●│●│●│●│ │                      ┌──────┼──────┐
│ └─┴─┴─┴─┴─┘ │                      ▼      ▼      ▼
└──────┬──────┘                   ┌─────┐┌─────┐┌─────┐
   ┌───┴───┐                      │Q1   ││Q2   ││Q3   │
   ▼       ▼                      └──┬──┘└──┬──┘└──┬──┘
┌─────┐ ┌─────┐                      ▼      ▼      ▼
│ C1  │ │ C2  │ (competing)       ┌────┐┌────┐┌────┐
└─────┘ └─────┘                   │ C1 ││ C2 ││ C3 │
                                  └────┘└────┘└────┘
```

**Popular Solutions:** RabbitMQ, Apache Kafka, AWS SQS, Redis Streams

## 4. WebSockets

Full-duplex, persistent connection for real-time bidirectional communication.

**Characteristics:**
- Upgrades from HTTP handshake to WS protocol
- Low latency (no request overhead per message)
- Stateful connection (server must track clients)

```
┌────────┐                              ┌────────┐
│ Client │                              │ Server │
└───┬────┘                              └───┬────┘
    │──── HTTP Upgrade: websocket ────────▶│
    │◀─── 101 Switching Protocols ─────────│
    │                                      │
    │══════════ WS Connection ═════════════│
    │                                      │
    │──────── {"type": "subscribe"} ──────▶│
    │◀─────── {"data": "update1"} ─────────│
    │◀─────── {"data": "update2"} ─────────│
    │──────── {"action": "ping"} ─────────▶│
    │◀─────── {"action": "pong"} ──────────│
    │                                      │
```

**Use Cases:** Chat apps, live dashboards, gaming, collaborative editing

## 4.2 Delivery Guarantees & Idempotency

**Delivery semantics** (often via queues/streams):
- **At-most-once**: Fast, but messages may be lost.
- **At-least-once**: No loss, but duplicates possible.
- **Exactly-once**: Hardest; requires coordination or idempotent consumers.

**Idempotency patterns:**
- Use **idempotency keys** (e.g., `Idempotency-Key` header) to dedupe.
- Store a **processed-events table** keyed by event id.
- Make operations **commutative** where possible (set-based writes).

**Correlation & tracing:**
- Include **correlation IDs** and **trace IDs** across services.
- Propagate context (W3C Trace Context) for observability.

## 4.1 Server-Sent Events (SSE) vs WebSockets

| Feature | SSE | WebSockets |
|---------|-----|------------|
| **Direction** | Server → Client | Bidirectional |
| **Protocol** | HTTP (text/event-stream) | WebSocket (ws/wss) |
| **Reconnection** | Built-in, automatic | App-managed |
| **Load Balancing** | Works with HTTP infrastructure | Requires sticky sessions or shared state |
| **Best For** | Live feeds, notifications, dashboards | Chat, multiplayer, collaboration |

**Rule of thumb:** Prefer **SSE** for server-push only and simpler infrastructure; use **WebSockets** when you need true two-way low-latency messaging.

## 5. Event-Driven Architecture (EDA)

Services communicate through events, enabling loose coupling and scalability.

**Core Concepts:**
- **Event**: Immutable fact about something that happened
- **Event Producer**: Emits events without knowing consumers
- **Event Consumer**: Reacts to events of interest
- **Event Broker**: Routes events (Kafka, EventBridge, etc.)

```
┌─────────────────────────────────────────────────────────────────┐
│                        EVENT BROKER                            │
│  ┌─────────────┬─────────────┬─────────────┬─────────────┐     │
│  │ orders.*    │ users.*     │ payments.*  │ inventory.* │     │
│  └─────────────┴─────────────┴─────────────┴─────────────┘     │
└───────────────────────────┬─────────────────────────────────────┘
         ▲                  │                  ▲
         │ publish          │ subscribe        │ publish
         │                  ▼                  │
    ┌─────────┐        ┌─────────┐        ┌─────────┐
    │ Order   │        │ Email   │        │ Payment │
    │ Service │        │ Service │        │ Service │
    └─────────┘        └─────────┘        └─────────┘
         │                  │                  │
    OrderCreated       (listens to        PaymentCompleted
    OrderCancelled      orders.*)         PaymentFailed
```

**Event Sourcing:** Store all changes as events, rebuild state by replay.

## 6. Pattern Selection Guide

```
Need real-time updates?  ──Yes──▶ WebSockets / SSE
       │
      No
       │
       ▼
Need guaranteed delivery? ──Yes──▶ Message Queue (RabbitMQ, SQS)
       │
      No
       │
       ▼
High-throughput streaming? ──Yes──▶ Event Streaming (Kafka)
       │
      No
       │
       ▼
Internal microservices?  ──Yes──▶ gRPC
       │
      No
       │
       ▼
Flexible client queries? ──Yes──▶ GraphQL
       │
      No
       │
       ▼
Public API / Simple CRUD ──────▶ REST
```

## 7. Quick Reference

| Pattern | When to Use | Watch Out For |
|---------|-------------|----------------|
| **Sync REST** | Simple CRUD, public APIs | Cascading failures, latency |
| **gRPC** | Low-latency internal calls | Browser support, debugging |
| **GraphQL** | Varied client data needs | N+1 queries, caching complexity |
| **Message Queue** | Decoupling, reliability | Message ordering, exactly-once |
| **WebSocket** | Real-time bidirectional | Connection management, scaling |
| **Event-Driven** | Loose coupling, audit trail | Eventual consistency, debugging |